<a href="https://colab.research.google.com/github/nicklausmillican/StatisticalRethinkingIISolutions/blob/main/StatisticalRethinkingSolutions2_Ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8

In [ ]:
install.packages(c("coda","mvtnorm","devtools","loo","dagitty"))
devtools::install_github("rmcelreath/rethinking@slim")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘numDeriv’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘abind’, ‘tensorA’, ‘distributional’, ‘Rcpp’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’, ‘checkmate’, ‘matrixStats’, ‘posterior’, ‘V8’





shape (NA -> 1.4.6.1) [CRAN]


Installing 1 packages: shape

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpZrn38E/remotes18c4c02219c/rmcelreath-rethinking-cbcb8ba/DESCRIPTION’ ... OK
* preparing ‘rethinking’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
* building ‘rethinking_2.13.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(rethinking)

Loading required package: parallel

rethinking (Version 2.13.2)


Attaching package: ‘rethinking’


The following object is masked from ‘package:stats’:

    rstudent




## Easy

### 8E1
#### Question
For each of the causal relationships below, name a hypothetical third variable that would lead to an interaction effect.  
> (1) Bread dough rises because of yeast.  

> (2) Education leads to higher income.  

> (3) Gasoline makes a car go.

#### Answers
> (1) (Had to google "How does yeast make bread rise")  Yeast gobbles up sugar in the dough, releasing gasses that are trapped in the matrix of the dough and causes rising.  This means that *sugar* is a third variable that would lead to an interaction; something like $rise = \alpha + ... + \beta_{y \times s} \times yeast \times sugar$.

> (2) Education may lead to higher income, depending on the area of specialization.  E.g., an investment banker will probably make more than a nurse.  So there is an interaction term like $income = \alpha + ... + \beta_{e \times s} \times education \times specialization$

> (3) Gas powers a car, contingent on the proper function of the rest of the car.  E.g., a car with no wheels will not move regardless of how much gas you give it.  $GO = \alpha + ... + \beta_{g \times w} \times gas \times functioning$


### 8E2-8E3
#### Question E2
Which of the following explanations invokes an interaction?  

> (1) Caramelizing onions requires cooking over low heat and making sure the onions do not dry out.  

> (2) A car will go faster when it has more cylinders or when it has a better fuel injector.  

> (3) Most people acquire their political beliefs from their parents, unless they get them instead from their friends.  

> (4) Intelligent animal species tend to be either highly social or have manipulative appendages (hands, tentacles, etc.).

#### Question E3
For each of the explanations in 8E2, write a linear model that expresses the stated relationship.

##### Answers
> (1) I don't know how carmelizing works, either.  Google tells me that it involves the non-enzymatic breakdown of sugar and the removal of water (which seems to contradict the question).  I think there's an interaction here since (I imagine, heat will not carmelize an onion if it's submerged in a pot of boiling water--so the effect of heat depends on moisture.  $$C = \beta_H H + \beta_M M + \beta_{HM} H M$$.  I don't have an intercept here because I imagine that no heat will produce no carmelizing.

> (2) I'm going to imagine that this could be an interactive effect: a car with more cylinders will perform better-than-linearly with more better fuel injectors.  In this case, $$S = \beta_C C + \beta_F F + \beta_{CF} C F$$.  But I could also imagine that any ole fuel injector will do the trick as long as it works, in which case, this is additive like $$S = \beta_C C + \beta_F F$$. Again, I don't have an intercept because a car with no cylinders probably won't go very fast...

> (3) This one is tricky.  It's describing an either-or situation.  Either a person gets their political beliefs from their parents or from their friends.  But this is NOT an interaction, as we've learned so far, since the effect of parents does not depend on the effect of friends.  We will later learn that the one's political beliefs may not be a linear function, and that this will cause interaction between parents and friends...just not interaction in the way we've learned so far.  So I'm going to say a linear model is like $$P = \alpha + \beta_I I$$, where $I$ is an indicator variable representing the source of influence (parents or friends).

> (4) No interaction here: the effect of sociality is not be described as being modulated by the effect of appendages (and vice versa).  $$I = \alpha + \beta_S S + \beta_A A$$